##  Реализация web-crawler для сбора коллекции документов с указанного ресурса  
Ресурс: HABR  
#### Требования:  
1.Объем собранной коллекции не менее 100 тысяч уникальных документов  
2.Реализация механизма очистки документов от не релевантной информации:  
    - html теги
    - ссылки на сторонние или внутренние ресурсы сайта
    - ссылки на изображения и видео
    - знаки препинания
3.Реализовать механизм выделения признаков документа:
    - автор
    - тематические теги
    - рейтинг
    - репосты
    - дата публикации
4.После проведенной обработки размер текста для каждого документа должен быть не менее 2000 символов  

#### Результат:
- код web-crawler
- инструмент очистки текстов
- исходная коллекция документов
- коллекция документов после очистки

# Алгоритм решения текущей задачи  
#### 1.Сбор информации  
Сначала была идея брать список всех статей на текущей странице, получать ссылки по тегу <а>, затем по ним делать запрос и получать необходимую информацию. На практике оказалось это очень долгим процессом, также возникали проблемы с количеством страниц, с которых собирались ссылки. В связи с этим была взята идея из статьи https://habr.com/ru/post/346198/, что можно просто перебрать нумерацию статей от 1 до 200 000. В процессе разработки было обнаружено, что данные собираются все равно достаточно долго. Тогда было решено ввести мультипроцессинг. Путем анализа на зависимость производительности от количества процессов было выбрано 7 процессов. Данные помещаются в папку /data в формате html страниц. В процессе сбора информации берется не вся страница, а лишь ее часть (все начиная с тега `<div class = 'post__text post__text-html post__text_v1'>` + информация о количестве лайков, дизлайков, комментариев, закладок).
#### 2.Очистка документа от нерелевантной информации  
Для очистки документа он сначало загружается с html файла (это происходит намного быстрее, чем если сразу загружать и очищать).
В ходе работы было обнаружено, что при использовании метода из библиотеки BeautifulSoap "text" кодовая часть статьи также считается за текст. В связи с этим используется cleanhtml. Далее очистка происходит с использованием регулярных выражений: убираются все знаки препинания, лишние отступы, цифры, ссылки и т.д. Помимо этого здесь происходит удаление стоп-слов (местоимения, предлоги, союзы) и лемминг.  
#### 3.Выделение признаков  
За признаки берется название `статьи(class = 'post__title-text'), автор статьи(class = 'user-info__nickname'),
дата публикации(class = 'post__time'), теги (class = 'post__tags-list'), рейтинг(class = 'voting-wjt__counter'), количество добавлений статьи в закладки(class = 'bookmark__counter'), количество просмотров статьи(class = 'post-stats__views-count'), количество комментариев(class = 'post-stats__comments-count')`.
#### 4.Запись отфильтрованной информации  
Проверяется количество слов (не менее 2000) и подходящие очищенные документы складываются в папку /data/processed

In [141]:
import requests
import numpy as np
from bs4 import BeautifulSoup
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pymorphy2
import os
from tqdm import tqdm
import codecs
import re
import multiprocessing
import os.path
import time
import pandas as pd
from pathlib import Path

In [147]:
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
print(BASE_DIR)

D:\Program Files\Anaconda3\Scripts\web-crawler


#### Первая попытка сбора данных

In [3]:
req = requests.get('https://habr.com/ru/hub/infosecurity/page1')
soup = BeautifulSoup(req.text)

In [4]:
soup = []
for page in range(1,10):
    #print('https://habr.com/ru/hub/infosecurity/page{}'.format(page))
    req = requests.get('https://habr.com/ru/hub/infosecurity/page{}'.format(page))
    articles = BeautifulSoup(req.text).findAll('a',{'class','post__title_link'}) #получили элементы статей
    soup.append([item['href'] for item in articles]) #получили ссылки

In [168]:
#исходная коллекция документов
articles_count = 0;
count = 100000;
required_amount = count+1;
PATH_FILE = "data";
i = 1
while(required_amount > 0):
    soup = []
    #перебираем все страницы (по 10 шт)
    for page in range(i,i+9):
        req = requests.get('https://habr.com/ru/hub/infosecurity/page{}'.format(page))
        articles = BeautifulSoup(req.text).findAll('a',{'class','post__title_link'}) #получили статьи
        soup.append([item['href'] for item in articles]) #получили ссылки на полную версию статей
    i+=10
    #получаем данные по каждой статье
    for i, article_link in enumerate(soup[0]):
        req = requests.get(article_link)
        article = BeautifulSoup(req.text).find('article',{'class':'post_full'}) #берем html текущей статьи
        data = BeautifulSoup(req.text).find('div',{'class':'post__text'}) #берем html текущей статьи
        metadata = BeautifulSoup(req.text).find('ul',{'class':'post-stats post-stats_post js-user_'}) #для определения количества символов
        if (len(data.text) > 2000):
            file = open(PATH_FILE + "\habr{}.html".format(articles_count),"w",encoding='utf-8');
            file.write(str(article));
            file.write(str(metadata));
            file.close();
            articles_count+=1;    
    #если количество статей < 100 000 
    required_amount = count - articles_count;


KeyboardInterrupt: 

### Сбор данных

In [169]:
def download_document(post_id):
    PATH_FILE = 'data'
    # выгрузка документа
    req = ''
    while req == '':
        try:
            req = requests.get('https://habrahabr.ru/post/' +str(post_id) + '/')
        except:
            print(post_id)
            print("Connection refused by the server..")
            print("Let me sleep for 5 seconds")
            print("ZZzzzz...")
            time.sleep(2)
            print("Was a nice sleep, now let me continue...")
            continue;
    # парсинг документа
    soup = BeautifulSoup(req.text, 'lxml') # instead of html.parser
    doc = {}
    data = soup.find('div',{'class':'post__text'}) #берем html текущей статьи
    if not soup.find("span", {"class": "post__title-text"}):
        return None;
    else:
        article = soup.find('article',{'class':'post_full'}) #берем html текущей статьи
        metadata = soup.find('ul',{'class':'post-stats post-stats_post js-user_'}) #для определения количества символов
        file_path = PATH_FILE + "\habr{}.html".format(post_id);
        if(os.path.exists(file_path)):
            return None;
        file = open(file_path,"w",encoding='utf-8');
        file.write(str(article));
        file.write(str(metadata));
        file.close();

In [ ]:
num_processor = 7
none_count = 0
with multiprocessing.Pool(num_processor) as p:
    docs = p.map(download_document, np.arange(200000))

#### Работа с одним конкретным файлом

In [35]:
PATH_FILE = "data";
file_name = "habr200078"
file = open(PATH_FILE + "\{}.html".format(file_name),"r",encoding='utf-8');
array = []
data = []
columns = (['FileName', 'Title', 'Author', 'Date', 'Rating','BookmarksAmount',
            'Tags', 'ViewsAmount', 'CommentsAmount','Text'])
document = BeautifulSoup(file.read())
document.find("pre").contents[0] = ''

for el in document.findAll("pre"):
    el.contents[0] = ""
for el in document.findAll("code"):
    el.contents[0] = ""

array = ([file_name, get_title(document), get_author(document), get_publication_date(document), get_rating(document), 
              get_bookmarks_count(document),
              get_tags(document),
              get_views_amount(document), 
              get_comments_amount(document),
              get_text_data(document)
             ])
data.append(dict(zip(columns, array)))


Статья устарела. В новой документации содержится самая актуальная информация из этого поста. См. Class.


Введение
Наследование
MK.Object
MK.Array
Matreshka.js v0.1
Matreshka.js v0.2
Реализация TodoMVC

Приветствую всех читателей и писателей Хабра.
В предыдущей статье мы поговорили об основах работы с Матрешкой. В этой, я хочу рассказать, как наследовать Матрешку и как строить пока что небольшие приложения на её базе.

Матрешка устроенна в виде класса, сконструтированного при помощи кастомной функции . Это немного измененная версия функции, о которой я писал на форуме javascript.ru (ссылка на доку).

Так почему классы? Класс — это лишь слово, не противоречащее парадигме прототипного программирования. Если взглянуть на документацию того же Backbone.js, то вы увидите, что и они оперируют словом «класс» без всяких стеснений. Мы можем поспорить о том, что в Javascript нет классов, есть конструкторы, и я с вами соглашусь, но, на деле, имеет ли этот спор смысл? Если конструктор выглядит как 

In [37]:
data[0]

{'Author': 'Finom',
 'BookmarksAmount': '45',
 'CommentsAmount': '2',
 'Date': '2013-10-29T22:58Z',
 'FileName': 'habr200078',
 'Rating': '+16',
 'Tags': ['Matreshka', 'Matreshka.js'],
 'Text': 'Статья устарела. В новой документации содержится самая актуальная информация из этого поста. См. Class.\n\n\nВведение\nНаследование\nMK.Object\nMK.Array\nMatreshka.js v0.1\nMatreshka.js v0.2\nРеализация TodoMVC\n\nПриветствую всех читателей и писателей Хабра.\nВ предыдущей статье мы поговорили об основах работы с Матрешкой. В этой, я хочу рассказать, как наследовать Матрешку и как строить пока что небольшие приложения на её базе.\n\nМатрешка устроенна в виде класса, сконструтированного при помощи кастомной функции . Это немного измененная версия функции, о которой я писал на форуме javascript.ru (ссылка на доку).\n\nТак почему классы? Класс — это лишь слово, не противоречащее парадигме прототипного программирования. Если взглянуть на документацию того же Backbone.js, то вы увидите, что и они оп

### Очистка документа

In [4]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    text = re.sub(cleanr, '', raw_html)
    return text

In [125]:
def clear_data(data):
    #ищем ссылки
    data = re.sub(r'(http|https)\S+',
                  '', data)
    data = re.sub(r'(http|https)\S+',
                  '', data)
    data = re.sub(r'(([a-zA-Z]*\.)*[a-zA-Z]*(\.com|\.io|\.ru|\.ua|\.en|\.gov|\.org|\.uk|\.us|.\/edu|\.net)\S+)','',data)
    data = re.sub(r'(([a-zA-Z]*\.)*[a-zA-Z]*(\.com|\.io|\.ru|\.ua|\.en|\.gov|\.org|\.uk|\.us|.\/edu|\.net))','',data)
    data.replace("\n","")
   # print("1//////////////////////////")
    #print(data)
    data.replace("\r","")
    #print("2///////////////////////")
    #print(data)
    
    #ищем знаки препинания
    data = re.sub(r',', '', data)
    #print("3///////////////////////////////")
    #print(data)
    data = re.sub(r'–|—|←|↑|→|↓|↔|↕|⇆|⇅', '', data)
    #print("4//////////////////////////////////////")
    #print(data)
    data = re.sub(r'\.', '', data)
    #print("5//////////////////////////////////////")
    #print(data)
    data = re.sub(r'\:', '', data)
    #print("6//////////////////////////////////////")
    #print(data)
    data = re.sub(r'\;', '', data)
    #print("7//////////////////////////////////////")
    #print(data)
    data = re.sub(r'\?', '', data)
    #print("8//////////////////////////////////////")
    #print(data)
    data = re.sub(r'\!', '', data)
    #print("9//////////////////////////////////////")
    #print(data)
    data = re.sub(r'\…', '', data)
    #print("10//////////////////////////////////////")
    #print(data)
    
    #ищем числа
    data = re.sub(r'[0-9]+', '', data)
    #print("11//////////////////////////////////////")
    #print(data)
    
    #ищем кавычки
    data = re.sub(r'\"|«|»|“|”', '', data)
    #print("12//////////////////////////////////////")
    #print(data)
    
    data = re.sub(r'\/|\\', '', data)
    #print("12//////////////////////////////////////")
    #print(data)
    data = re.sub(r'\(@[a-z]*\)', '', data)
    #print("13//////////////////////////////////////")
    #print(data)
    
    #убираем скобки
    data = re.sub(r'\(|\)','',data)
    #print("14//////////////////////////////////////")
    #print(data)
    
    #убираем специальные символы хабра
    data = re.sub(r'Q|UPD','',data)
    #print("14//////////////////////////////////////")
    #print(data)
    
    #убираем лишние пробелы
    data = re.sub(r'\s\n',' ',data)
    #print("15//////////////////////////////////////")
    #print(data)
    data = re.sub(r'[\s]{2,}',' ',data)
    #print("16//////////////////////////////////////")
    #print(data)
    
    #выравниваем
    data = re.sub(r'\n|\r',' ',data)
    #print("17//////////////////////////////////////")
    #print(data)
    data = data.lower()
    return data

In [44]:
new_data = clear_data(data[0]['Text'])

1//////////////////////////
Статья устарела. В новой документации содержится самая актуальная информация из этого поста. См. Class.


Введение
Наследование
MK.Object
MK.Array
Matreshka.js v0.1
Matreshka.js v0.2
Реализация TodoMVC

Приветствую всех читателей и писателей Хабра.
В предыдущей статье мы поговорили об основах работы с Матрешкой. В этой, я хочу рассказать, как наследовать Матрешку и как строить пока что небольшие приложения на её базе.

Матрешка устроенна в виде класса, сконструтированного при помощи кастомной функции . Это немного измененная версия функции, о которой я писал на форуме  (ссылка на доку).

Так почему классы? Класс — это лишь слово, не противоречащее парадигме прототипного программирования. Если взглянуть на документацию того же Backbone.js, то вы увидите, что и они оперируют словом «класс» без всяких стеснений. Мы можем поспорить о том, что в Javascript нет классов, есть конструкторы, и я с вами соглашусь, но, на деле, имеет ли этот спор смысл? Если конструкто

In [39]:
print(new_data)

статья устарела в новой документации содержится самая актуальная информация из этого поста см class введение наследование mkobject mkarray matreshkajs v matreshkajs v реализация todomvc приветствую всех читателей и писателей хабра в предыдущей статье мы поговорили об основах работы с матрешкой в этой я хочу рассказать как наследовать матрешку и как строить пока что небольшие приложения на её базе матрешка устроенна в виде класса сконструтированного при помощи кастомной функции это немного измененная версия функции о которой я писал на форуме ссылка на доку так почему классы класс это лишь слово не противоречащее парадигме прототипного программирования если взглянуть на документацию того же backbonejs то вы увидите что и они оперируют словом класс без всяких стеснений мы можем поспорить о том что в javascript нет классов есть конструкторы и я с вами соглашусь но на деле имеет ли этот спор смысл если конструктор выглядит как класс плавает как класс и крякает как класс то это наверное и е

In [36]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [9]:
def set_stop_words(stopWordsList, words):
    for word in words:
        stopWordsList.add(word)

In [10]:
stopWords = set(stopwords.words('russian'))
new_stop_words = ["это","лишь","наш","ваш","ещё","уже","как","чтобы","а","но","если","что","нибудь","ваш","когда",
                  "где","зачем","почему","иначе","поэтому","потому","разве","при","после"]
set_stop_words(stopWords,new_stop_words)  

In [11]:
print(stopWords)

{'вы', 'тоже', 'через', 'вот', 'ни', 'до', 'впрочем', 'зачем', 'их', 'всех', 'такой', 'была', 'ещё', 'вдруг', 'а', 'уж', 'тебя', 'всю', 'но', 'моя', 'будет', 'есть', 'ли', 'при', 'будто', 'надо', 'себя', 'эти', 'о', 'быть', 'были', 'все', 'этом', 'другой', 'я', 'того', 'один', 'об', 'чего', 'после', 'тут', 'он', 'ж', 'чтобы', 'потом', 'два', 'поэтому', 'ты', 'ее', 'него', 'почему', 'это', 'что', 'всего', 'тем', 'три', 'к', 'можно', 'ничего', 'со', 'этой', 'этот', 'у', 'бы', 'мой', 'свою', 'больше', 'лучше', 'от', 'на', 'здесь', 'том', 'над', 'тот', 'какой', 'почти', 'конечно', 'нельзя', 'какая', 'них', 'ней', 'чуть', 'для', 'сейчас', 'было', 'всегда', 'эту', 'только', 'так', 'они', 'ним', 'более', 'иначе', 'когда', 'даже', 'ну', 'ей', 'чем', 'хоть', 'и', 'нас', 'совсем', 'хорошо', 'опять', 'разве', 'там', 'кто', 'мы', 'потому', 'теперь', 'если', 'ведь', 'из', 'мне', 'в', 'то', 'перед', 'может', 'нибудь', 'этого', 'за', 'тогда', 'ваш', 'сам', 'не', 'где', 'или', 'нет', 'между', 'вас', '

In [40]:
morph = pymorphy2.MorphAnalyzer()
results = [morph.parse(word)[0].normal_form for word in new_data.split(' ')] #лемминг
results = [x for x in results if len(x)>1] #рассматривать слова размера 1 нет смысла 

In [13]:
print(results)

['статья', 'устарелый', 'новый', 'документация', 'содержаться', 'самый', 'актуальный', 'информация', 'из', 'это', 'пост', 'сантиметр', 'class', 'введение', 'наследование', 'mkobject', 'mkarray', 'matreshkajs', 'matreshkajs', 'реализация', 'todomvc', 'приветствовать', 'весь', 'читатель', 'писатель', 'хабра', 'предыдущий', 'статья', 'мы', 'поговорить', 'основа', 'работа', 'матрёшка', 'этот', 'хотеть', 'рассказать', 'как', 'наследовать', 'матрёшка', 'как', 'строить', 'пока', 'что', 'небольшой', 'приложение', 'на', 'её', 'база', 'матрёшка', 'устроенный', 'вид', 'класс', 'сконструтированный', 'при', 'помощь', 'кастомный', 'функция', 'это', 'немного', 'изменить', 'версия', 'функция', 'который', 'писать', 'на', 'форум', 'ссылка', 'на', 'доку', 'так', 'почему', 'класс', 'класс', 'это', 'лишь', 'слово', 'не', 'противоречащий', 'парадигма', 'прототипный', 'программирование', 'если', 'взглянуть', 'на', 'документация', 'тот', 'же', 'backbonejs', 'то', 'вы', 'увидеть', 'что', 'они', 'оперировать', 

In [41]:
#исключение стоп слов
filtered_data = []
for w in results:
    if w not in stopWords:
        filtered_data.append(w)

In [42]:
filtered_data

['статья',
 'устарелый',
 'новый',
 'документация',
 'содержаться',
 'самый',
 'актуальный',
 'информация',
 'пост',
 'сантиметр',
 'class',
 'введение',
 'наследование',
 'mkobject',
 'mkarray',
 'matreshkajs',
 'matreshkajs',
 'реализация',
 'todomvc',
 'приветствовать',
 'весь',
 'читатель',
 'писатель',
 'хабра',
 'предыдущий',
 'статья',
 'поговорить',
 'основа',
 'работа',
 'матрёшка',
 'хотеть',
 'рассказать',
 'наследовать',
 'матрёшка',
 'строить',
 'пока',
 'небольшой',
 'приложение',
 'её',
 'база',
 'матрёшка',
 'устроенный',
 'вид',
 'класс',
 'сконструтированный',
 'помощь',
 'кастомный',
 'функция',
 'немного',
 'изменить',
 'версия',
 'функция',
 'который',
 'писать',
 'форум',
 'ссылка',
 'доку',
 'класс',
 'класс',
 'слово',
 'противоречащий',
 'парадигма',
 'прототипный',
 'программирование',
 'взглянуть',
 'документация',
 'backbonejs',
 'увидеть',
 'оперировать',
 'слово',
 'класс',
 'всякий',
 'стеснение',
 'мочь',
 'поспорить',
 'javascript',
 'класс',
 'конструк

### Выделение признаков и текста для дальнейшего анализа

In [76]:

def get_title(doc):
    try:
        return doc.find('span',{'class', 'post__title-text'}).text
    except:
        return None
#автор
def get_author(doc):
    try:
        return doc.find('span',{'class', 'user-info__nickname'}).text
    except:
        return None
#тематические теги
def get_tags(doc):
    try:
        tags = doc.find('dd',{'class', 'post__tags-list'}).findAll('li')
        tags_list = list()
        for tag in tags:
            tags_list.append(tag.text)
        return tags_list  
    except:
        return None
    
#рейтинг
def get_rating(doc):
    try:
        rating = doc.find('span',{'class', 'voting-wjt__counter'}).text
        return rating
    except:
        return None

#закладки
def get_bookmarks_count(doc):
    try:
        return doc.find('span',{'class', 'bookmark__counter'}).text
    except:
        return None

#дата публикации
def get_publication_date(doc):
    try:
        return doc.find('span',{'class', 'post__time'})['data-time_published']
    except:
        return None
#другая метаинформация, предоставляемая ресурсом
#количество просмотров
def get_views_amount(doc):
    try:
        return doc.find('span',{'class', 'post-stats__views-count'}).text
    except:
        return None
def get_comments_amount(doc):
    try:
        return doc.find('span',{'class', 'post-stats__comments-count'}).text;
    except:
        return None

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    text = re.sub(cleanr, '', raw_html)
    return text

def get_text_data(doc):
    try:
        text = str(doc.find('div',id = 'post-content-body'))
        return cleanhtml(text)
    except:
        return None


### Запись документа после очистки

In [45]:
data[0]['Text'] = str(filtered_data);
df = pd.DataFrame(data[0])

PATH_FILE = 'filtered-data'
print(len(filtered_data))
if(len(filtered_data) > 2000):
    df.to_csv("results.csv")
    file = open(PATH_FILE + "\habr200078.txt","w",encoding='utf-8');
    file.write(str(data[0]['Text']));
    file.close();

620


#### Работа с несколькими документами

In [57]:
fds = sorted(os.listdir('data/test'))
for file in fds:
    file_name = file.split(".")[0]
    print(file_name)

habr1
habr10
habr100
habr101
habr9


In [79]:
def load_data():
    columns = (['FileName', 'Title', 'Author', 'Date', 'Rating','BookmarksAmount',
                'Tags', 'ViewsAmount', 'CommentsAmount','Text'])
    data = []
    fds = sorted(os.listdir('data/test'))
    for file in fds:
        array = []
        
        file_name = file.split(".")[0]
        print(file_name)
        file_data = open('data/test/{}'.format(file),"r",encoding='utf-8');

        document = BeautifulSoup(file_data.read())
        
        #убираем части кода в тексте
        if(document.find("pre")!=None):
            document.find("pre").contents[0] = ''
            for el in document.findAll("pre"):
                el.contents[0] = ""
            for el in document.findAll("code"):
                el.contents[0] = ""

        array = ([file_name, get_title(document), get_author(document), get_publication_date(document), get_rating(document), 
                      get_bookmarks_count(document),
                      get_tags(document),
                      get_views_amount(document), 
                      get_comments_amount(document),
                      get_text_data(document)
                     ])
        data.append(dict(zip(columns, array)))
    return data


In [85]:
def filter_data(data):
    filtered_data = []
    for w in data:
        if w not in stopWords:
            filtered_data.append(w)
    return filtered_data

In [129]:
def write_data(filtered_data,data,words,final_data):
    data['Text'] = words;
    PATH_FILE = 'filtered-data'
    print(len(filtered_data))
    if(len(filtered_data) > 2000):
        final_data.append(data)
        file = open(PATH_FILE + "\{}.txt".format(data['FileName']),"w",encoding='utf-8');
        file.write(filtered_data);
        file.close();

In [138]:
data = load_data()

habr1
habr242359
habr242361
habr242381
habr242385
habr242387


In [139]:
data

[{'Author': 'deniskin',
  'BookmarksAmount': '8',
  'CommentsAmount': '62',
  'Date': '2006-07-13T14:23Z',
  'FileName': 'habr1',
  'Rating': '+1',
  'Tags': ['хабрахаб', 'wiki', 'FAQ', 'ЧАВО', 'механизм', 'движок'],
  'Text': 'Привет!\n\n\nДля сборника ответов на Часто Задаваемые Вопросы мы решили использовать идею wiki, поскольку, как нам кажется, нет смысла писать одному конкретному человеку FAQ для такого сайта, как Хабрахабр. Все равно останутся люди, у которых останутся вопросы.\n\nПосему, обращаюсь к читателям группы со следующим вопросом: какой wiki-движок из доступных, по-вашему, идеально подходит для составления «коллективного» ЧАВО по сайту?\n',
  'Title': 'Wiki-FAQ для Хабрахабра',
  'ViewsAmount': '30,9k'},
 {'Author': 'nitro2005',
  'BookmarksAmount': '158',
  'CommentsAmount': '17',
  'Date': '2014-11-05T09:41Z',
  'FileName': 'habr242359',
  'Rating': '+11',
  'Tags': ['zeromq', 'sockets', 'сокеты', 'клиент-сервер', 'сервисы'],
  'Text': 'В любом среднем или крупном при

#### Очистка

In [140]:
final_data = []
for document in data:
    new_data = clear_data(document['Text']) #получаем текст статьи
    results = [morph.parse(word)[0].normal_form for word in new_data.split(' ')] #лемминг
    results = [x for x in results if len(x)>1] #рассматривать слова размера 1 нет смысла 
    
    #убираем стоп слова (англ и русские)
    stopWords = set(stopwords.words('russian'))
    filtered_data = filter_data(results)
    stopWords = set(stopwords.words('english'))
    filtered_data = filter_data(filtered_data)
    
    filtered_text = ' '.join(filtered_data)
    write_data(filtered_text,document,filtered_data,final_data) #записываем отформатированный текст
    
df = pd.DataFrame(final_data)
df.head()
df.to_csv("results.csv")

325
8821
10501
7780
8806
5887
